In [15]:
import pickle
import torch
from src.model import Transformer, TransformerConfig
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [63]:
model_folder = "run_data"#"models"

with open(model_folder+"/config.json", "r") as f:
    settings = json.loads(f.read())

with open(model_folder+"/word_data.json", "r") as f:
    word_data = json.loads(f.read())

word_to_index = {k:int(v) for k, v in word_data["word_to_index"].items()}
index_to_word = {int(k):v for k, v in word_data["index_to_word"].items()}

In [42]:
# import torch
# import seaborn as sns

# initial_lr = 0.001
# max_lr = 0.004
# min_lr = 0.0001
# optimizer = torch.optim.Adam([torch.Tensor([[1, 2]]), torch.Tensor([[1]])], lr=initial_lr)

# total_epochs = 200
# warmup_steps = 40
# scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=initial_lr/max_lr, total_iters=warmup_steps)
# scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = total_epochs-warmup_steps, eta_min = min_lr)
# scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[warmup_steps])

# lrs = []
# for i in range(total_epochs):
#     lr = scheduler.optimizer.param_groups[0]['lr']
#     lrs.append(lr)
#     scheduler.step()

# sns.lineplot(lrs)

In [64]:
config = TransformerConfig(vocab_size=settings["vocabulary_size"], max_input_length=settings["max_input_length"], num_heads=settings["num_heads"], num_blocks=settings["num_blocks"], embedding_size=settings["embedding_size"])
transformer = Transformer(config)
transformer.load_state_dict(torch.load(model_folder+"/model.pt", map_location=torch.device(device)))
transformer.to(device)
transformer.train()
print("Model loaded")

Model loaded


In [74]:
def generate_next_token(tokens=None, sample=False):
    if tokens == None:
        tokens = [len(index_to_word)-1]
    x = torch.LongTensor([tokens]).to(device)
    with torch.no_grad():
        y = transformer(x)
    # Don't allow the model to generate <unknown> tokens
    y = torch.nn.functional.softmax(y[:, -1, :y.shape[2]-1].view(-1).detach().cpu(), dim=0)
    if not sample:
        pred = y.argmax(dim=-1).view(-1)
    else:
        dist = torch.distributions.categorical.Categorical(probs=y)
        pred = dist.sample([1])
    next_word = pred.item()
    return next_word

def print_sentence(words):
    print(" ".join([index_to_word[word] for word in words]))

def generate_sentence(start=None, sample=False, length=50):
    if start == None:
        sentence = []
    else:
        words = start.split(" ")
        sentence = [word_to_index[x] for x in words]
    
    while len(sentence)<length:
        if len(sentence) < settings["max_input_length"]:
            next_word = generate_next_token(sentence, sample)
            sentence += [next_word]
        else:
            next_word = generate_next_token(sentence[-settings["max_input_length"]:], sample)
            sentence += [next_word]
    
    print_sentence(sentence)


In [81]:
transformer.train()
for i in range(5):
    generate_sentence("red like the sun", False, 50)

red like the sun , and not a place , but that the very tongue of a man , that he may be a fool . i am a fool , and a man , and a man , a very honest , a very gallant one , and a
red like the sun , and make a full of the sun , the moon , the sun , the sky , and the moon , the moon , the sun , the sky , the moon , the moon , the sky , the moon , the moon ,
red like the sun , and the wind will not be my flesh . i am a man , and i will not be a man . exit . scene iv . a room in capulets house . enter benvolio , pyramus and thisbe . thisbe . i have met
red like the sun , and in the night the sun , and the sun , to be the sun , the sun , the moon , the moon , the moon , the moon , the sun , and the moon , the moon , the sky , the
red like the sun , and with a little wind , and then the wind is the sun , the moon , the moon , the wind , the moon , the wind , the moon , the moon , the moon , the moon , the moon , the


In [94]:
transformer.train()
for i in range(5):
    generate_sentence("mine", False, 50)

mine , and i , and my love , and my love , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my
mine own . i will not be merry , for i am sure i have a wife , and i will not be glad to see a man . i am glad i am a gentleman , and i will be sworn to the king . i am a king
mine , and my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart , my heart
mine honour . i will not trust you . i will be your servant . exit . king . i am sorry i am glad you have a woman . rosaline . i am a poor and a man , and i am a gentleman of a gentleman , and
mine own . i am not mad ; i am not mad . i am not mad . i am a gentleman of all the world , and i am sure i am a gentleman , and i am sure i am not i am a gentleman of the king


In [75]:
generate_sentence("hark")
generate_sentence("through yonder")
generate_sentence("through yonder")
generate_sentence("through yonder")
generate_sentence("may")
generate_sentence("may")
generate_sentence("may")
generate_sentence("and")
generate_sentence("and")
generate_sentence("and")
generate_sentence("i")
generate_sentence("i")
generate_sentence("i")

hark , what noise is this ? enter a messenger . messenger . the queen is gone . messenger . my lord , i have sent for you . the news is gone from rome . messenger . the queen is coming . cleopatra . i have heard the lady
through yonder , and in the world , i will not break the fire . exit . scene iii . the same . a room in the palace . enter king , fluellen , fluellen , lords and captain bishop . king . i am glad of the king .
through yonder , and then he is , and i am sure he is not . i am sure , he is not here . if he be sure , and yet i will not be satisfied . if he be not , ill be sworn . if he be
through yonder field , and in the world , i will not be bold to be a man . but , good night , good night , and i will not be a messenger , for my sake , i am glad to see a man . i am glad
may be a little better than a man . i am a gentleman , and i will be sworn to be a soldier , and i will be rid of a man . i will be glad to be a gentleman of the world . exeunt . scene iii .
may be well 

Best of:

- [if thou didst] not , thou art a fool . i am a fool , and thou art a fool . thou art a fool . clown . i am
- [i] have a woman , and i have a good heart . but i will be a man of mine , and i will not be satisfied .
- [i] am a man , and i am sure of it . i have said , and i will not be sworn to thee . i am a king ,
- [you are] a merciful old wench . i am not able to wait upon my friend . i am glad to have merry a launcelet , if i had a
- [you are] a beauteous blossom , and i will encounter thee with my nan . i am a proper friend . anne . i would not play a fool ,
- i am not drunk for certain , nor i am not of any part of my sex . i am sorry for neither
- [you are] in the field . enter a messenger . messenger . news , madam , news ! cleopatra . what news ? messenger . the news ? messenger . the king is gone . messenger . the lord mortimer is coming . antony . the noble antony is come
- [like the sky] , and the wind and the moon . o , what a one that is in the other ?